### Gradient of Mean, Var and Std

In [47]:
from cs231n.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
import numpy as np

data = np.arange(1.0, 9.0)
N = len(data) + 1


def data_append(f):
    def warpper(x):
        global data
        data = np.append(data, x)
        ret = f(x)
        data = data[:-1]
        return ret

    return warpper


@data_append
def hat(x):
    """
    Args:
        x (num): num be appended to data

    Returns:
        (ndarray): hat of new_data, shape likes new_data [1XN]
    """
    mean = np.mean(data)
    std = np.std(data)
    return (data - mean) / std


@data_append
def mean(x):
    return np.mean(data)


@data_append
def var(x):
    return np.var(data)


@data_append
def std(x):
    return np.std(data)


def rel_error(x, y):
    """Returns relative error."""
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [48]:
def d_mean(x):
    return 1 / N


def d_var(x):
    return 2 * (x - mean(x)) / N


def d_std(x):
    return (x - mean(x)) / (N * std(x))


def d_hat(x):
    m = mean(x)
    s = std(x)
    dh_cors = (((data - m) * (x - m) / (s**2)) + 1) / (-1 * N * s)
    dh_self = (N - 1 - hat(x)[-1] ** 2) / (s * N)
    return np.append(dh_cors, dh_self).sum()

In [49]:
x = np.array([9.0])
for str in ["mean", "var", "std", "hat"]:
    f = eval(str)
    d = eval(f"d_{str}")
    f_eval = eval_numerical_gradient_array if str == "hat" else eval_numerical_gradient
    print(f"{str}:{f(x)}")
    numerical = f_eval(f, x, h=0.01)
    print(numerical, d(x[0]))

mean:5.0
[0.11111111] 0.1111111111111111
var:6.666666666666667
[0.88888889] 0.8888888888888888
std:2.581988897471611
[0.1721325] 0.17213259316477408
hat:[-1.54919334 -1.161895   -0.77459667 -0.38729833  0.          0.38729833
  0.77459667  1.161895    1.54919334]
[-6.83047369e-14] 2.7755575615628914e-17
